In [15]:
import funcoes as fc
import numpy as np
import os
import pandas as pd
import pyautogui
import pyperclip
import time
import tkinter as tk
import undetected_chromedriver as uc
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tabulate import tabulate
from tkinter import scrolledtext
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Edge
from selenium.webdriver import Chrome

In [16]:
#INSTALA O CHROME DRIVEr MAIS ATUALIZADO
# Desativa o bloqueio de pop-ups
# options = webdriver.ChromeOptions()
# options.add_argument("--disable-popup-blocking") 
#DEFINE O TEMPO DE EXECUÇÃO PARA CADA COMANDO DO PYAUTOGUI
pyautogui.PAUSE = 0.7
#INICIA O NAVEGADOR
# Caminho do ChromeDriver local
# chrome_driver_path = r"main\chromedriver-win64\chromedriver.exe"  # Altere para o caminho correto do seu ChromeDriver
# Configura o serviço do ChromeDriver
# servico = Service(ChromeDriverManager().install())
# Inicia o navegador Chrome
# navegador = webdriver.Chrome(service=servico, options=options)
navegador = Edge()
navegador.maximize_window()
#ENTRA NO SEI
navegador.get('https://sei.anatel.gov.br/')


In [37]:
hoje = datetime.now().strftime("%d/%m/%y")
despacho_atual = "Despachos para Drones aprovados "+hoje
print(despacho_atual)

Despachos para Drones aprovados 20/03/25


In [ ]:
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrConteudoVisualizacao')
time.sleep(1)
#CLICA NO ICONE DE LEGO
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/bloco_incluir_protocolo.svg?18')]")
#navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]').click()
#SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
time.sleep(1.5)
navegador.switch_to.frame('ifrVisualizacao')
select_element = navegador.find_element(By.ID, 'selBloco')
select = Select(select_element)
#PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
for opcao in select.options:
    print(opcao.text.lower(), type(opcao.text))
    if "despachos para drones aprovados "+hoje in opcao.text.lower():
        select.select_by_visible_text(opcao.text)
        print("Despacho achado")
        despacho_achado = True
        break
if despacho_achado:
    print("Faz tudo")
else:
    print("Despacho do dia não foi achado. Crie o despacho e inclua manualmente no bloco. ")



  <class 'str'>
354285 - para assinatura do davison <class 'str'>
354281 - despachos para drones aprovados 20/03/25 <class 'str'>
Despacho achado


In [56]:
inclui_despacho(navegador, '53500.017811/2025-31')

In [ ]:
def inclui_despacho(navegador, processo):    
    #INCLUI DESPACHO NO BLOCO
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrArvore')
    #CLICA NO DESPACHO DECISORIO
    fc.clica_noelemento(navegador, By.PARTIAL_LINK_TEXT,"Despacho Decisório")
    #navegador.find_element(By.PARTIAL_LINK_TEXT, "Despacho Decisório").click()
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    time.sleep(1)
    #CLICA NO ICONE DE LEGO
    fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/bloco_incluir_protocolo.svg?18')]")
    #navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]').click()
    #SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
    time.sleep(1.5)
    navegador.switch_to.frame('ifrVisualizacao')
    select_element = navegador.find_element(By.ID, 'selBloco')
    select = Select(select_element)
    #PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
    despacho_achado = False
    hoje = hoje = datetime.now().strftime("%d/%m/%y")
    for opcao in select.options:
        if "despachos para drones aprovados "+hoje in opcao.text.lower():
            select.select_by_visible_text(opcao.text)
            despacho_achado = True
            break
    if not despacho_achado:
        while True:
            opcao = input("Pode ser que o despacho nao foi achado. Crie o despacho (abrindo uma nova aba) e digite [1]. Caso houve algum problema e seja necessário incluir manualmente, digite [2]: ")
            if opcao == '1':
                inclui_despacho(navegador, processo)
                return
            elif opcao == '2':
                return
            else:
                print("Opção inválida. Tente novamente.")
    time.sleep(0.5)
    #CLICA NO BOTAO DE INCLUIR NO BLOCO
    fc.clica_noelemento(navegador, By.XPATH,'//*[@id="sbmIncluir"]')
    #navegador.find_element(By.XPATH, '//*[@id="sbmIncluir"]').click()
    #VOLTA PARA PAGINA INICIAL DO PROCESSO
    # navegador.switch_to.default_content()
    # navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo)
    # elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
    # elementos.send_keys(Keys.ENTER)
    fc.vai_para_processo(navegador, processo)
    # time.sleep(3)
    fc.muda_para_iframe(navegador, By.ID, 'ifrConteudoVisualizacao')
    # navegador.switch_to.frame('ifrConteudoVisualizacao')
    #ADICIONA NOTA PARA AGUARDAR ASSINATURA
    #CLICA NO ICONE DE ANOTACAO
    fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/anotacao_cadastro.svg?18')]")
    #clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[16]')
    time.sleep(0.1)
    #INSERE O TEXTO DA ANOTACAO
    navegador.switch_to.frame('ifrVisualizacao')
    navegador.find_element(By.ID, 'txaDescricao').send_keys('Aguardando assinatura.')
    #SALVA O TEXTO
    navegador.find_element(By.NAME, 'sbmRegistrarAnotacao').click()
    #DA COMO APROVADO E ANOTA NO TXT DE PROCESSOS CONFORMES

In [ ]:
navegador.switch_to.window(navegador.window_handles[0])

In [ ]:
processo = '53500.001150/2025-22'
fc.vai_para_processo(navegador, processo)
time.sleep(1)
#PEGA DADOS DA DECLARACAO DE CONFORMIDADE
navegador.switch_to.frame('ifrArvore')

#CONFERE SE EXISTE A DECLARACAO DE CONFORMIDADE OU UMA PASTA
if fc.check_element_exists(By.XPATH, '//*[@id="spanPASTA1"]', navegador):
    navegador.find_element(By.XPATH, '//*[@id="spanPASTA1"]').click()
    time.sleep(1)
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
    time.sleep(0.3)

elif fc.check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade", navegador):
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
    time.sleep(0.3)

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrConteudoVisualizacao')
navegador.switch_to.frame('ifrVisualizacao')

#ARMAZENA EMAIL DO SOLICITANTE
emailSol = navegador.find_element(By.XPATH, '/html/body/div[3]/a[1]').text

In [ ]:
counter = 0
while True:
    fc.muda_janela('Distribuição Processo Drone.xlsx')
    pyautogui.hotkey('ctrl', 'c')
    time.sleep(0.5)
    pyautogui.hotkey('ctrl', 'c')
    time.sleep(0.2)
    n_processo = pyperclip.paste()
    if n_processo == '':
        counter += 1
    if counter == 5:
        print('Parece que o programa chegou ao fim da iteração')
        navegador.close()
        break
    fc.muda_janela('Distribuição Processo Drone.xlsx')
    pyautogui.press('down')

Parece que o programa chegou ao fim da iteração


In [ ]:
def abrir_janela():
    janela = tk.Toplevel()
    janela.title("Nova Janela")
    janela.geometry("300x200")

    # Garante que a nova janela fique em foco e no topo
    janela.attributes('-topmost', 1)  # Mantém a janela no topo
    janela.lift()                     # Eleva a janela
    janela.focus_force()              # Força o foco na janela

    tk.Label(janela, text="Essa janela está em foco!").pack(pady=20)
    tk.Button(janela, text="Fechar", command=janela.destroy).pack()

# Janela principal
root = tk.Tk()
root.title("Janela Principal")
root.geometry("300x200")

tk.Button(root, text="Abrir Nova Janela", command=abrir_janela).pack(pady=50)

root.mainloop()

In [ ]:

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
lista_despachos = navegador.find_elements(By.PARTIAL_LINK_TEXT, 'Despacho Decisório')
for i in lista_despachos:
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrArvore')
    i.click()
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    navegador.switch_to.frame('ifrVisualizacao')
    time.sleep(1)
    if fc.check_element_exists(By.XPATH, "//*[contains(text(), 'Revogar o')]", navegador):
        navegador.switch_to.default_content()
        navegador.switch_to.frame('ifrConteudoVisualizacao')
        print('processo achado')
        time.sleep(1)
        #CLICA NO ICONE DE LEGO
        fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/bloco_incluir_protocolo.svg?18')]")
        
        #navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]').click()
        #SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
        time.sleep(1.5)
        navegador.switch_to.frame('ifrVisualizacao')
        select_element = navegador.find_element(By.ID, 'selBloco')
        select = Select(select_element)
        #PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
        for opcao in select.options:
            if "Despachos para Drones aprovados" in opcao.text:
                select.select_by_visible_text(opcao.text)
                break
        time.sleep(0.5)
        #CLICA NO BOTAO DE INCLUIR NO BLOCO
        fc.clica_noelemento(navegador, By.XPATH,'//*[@id="sbmIncluir"]')
        #navegador.find_element(By.XPATH, '//*[@id="sbmIncluir"]').click()
        



processo achado


In [ ]:
#VOLTA PARA PAGINA INICIAL DO PROCESSO
navegador.switch_to.default_content()
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys('53500.047484/2024-61')
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)
navegador.switch_to.frame('ifrConteudoVisualizacao')
time.sleep(1)
#ADICIONA NOTA PARA AGUARDAR ASSINATURA
#CLICA NO ICONE DE ANOTACAO
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/anotacao_cadastro.svg?18')]")
#clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[16]')
time.sleep(0.1)
#INSERE O TEXTO DA ANOTACAO
navegador.switch_to.frame('ifrVisualizacao')
navegador.find_element(By.ID, 'txaDescricao').send_keys('Pedido de Cancelamento')
#SALVA O TEXTO
navegador.find_element(By.NAME, 'sbmRegistrarAnotacao').click()
#DA COMO APROVADO E ANOTA NO TXT DE PROCESSOS CONFORMES

In [2]:
import funcoes as fc
fc.preenche_planilhageral('53500.000742/2025-27', 'André Jacinto Rodrigues', 'Não', 'Aprovado', '', 'Rafael Basilio Chaves')

Célula encontrada: 53500.000742/2025-27
Parece que a janela 'Google Chrome' não está visível para o programa.
Parece que a janela 'theomation' não está visível para o programa.
